# IIJA All Projects 

In [1]:
# ! pip install nltk
# ! pip install textblob

In [2]:
import numpy as np
import pandas as pd
from siuba import *

from shared_utils import geography_utils
from dla_utils import _dla_utils

from calitp import to_snakecase

import utils

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [3]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)


## Read Data

In [4]:
proj = utils.read_data_all()

In [5]:
proj.head(1)

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value
0,2022-03-03,Y230,STBG-URBANIZED >200K IIJA,5002174,0314000222L,BELL AVE. FROM BOLLENBACHER AVE. TO ASTORIA ST ROAD REHABILITATION (TC),67.00,Cong Dist 6,4.00,4R - No Added Capacity,"2,841,397.00",L5002SACOG


In [6]:
len(proj)

1241

* need 1241 projects. 

In [7]:
## agency identifier column doesnt always have locode

## Redo `identify_agency` function

In [8]:
#check why we are getting more rows

#utils.add_name_from_locode(proj, 'summary_recipient_defined_text_field_1_value')

#utils.identify_agency(proj, 'summary_recipient_defined_text_field_1_value')


In [9]:
# locode_proj = ((proj[~proj['summary_recipient_defined_text_field_1_value'].str.contains(" ")]))
# locode_proj = locode_proj>>filter(_['summary_recipient_defined_text_field_1_value']!='None')
    

In [10]:
# locode_proj = (utils.add_name_from_locode(locode_proj, 'summary_recipient_defined_text_field_1_value'))

In [11]:
# len(locode_proj)

In [12]:
# no_locode = ((proj[proj['summary_recipient_defined_text_field_1_value'].str.contains(" ")]))
# no_entry = proj>>filter(_['summary_recipient_defined_text_field_1_value']=='None')

In [13]:
# print(len(no_locode))
# print(len(no_entry))

In [14]:
# no_locode = pd.concat([no_locode, no_entry])

In [15]:
# len(no_locode)

In [16]:
#     #add county codes:
# county_base = to_snakecase(pd.read_excel(f"gs://calitp-analytics-data/data-analyses/dla/dla-iija/Copy of County.xlsx", sheet_name='County', header=[1]))
# county_base.drop(columns =['unnamed:_0', 'unnamed:_4'], axis=1, inplace=True)
# county_base['county_description'] = county_base['county_description'] + " County"

In [17]:
# locodes = to_snakecase(pd.read_excel(f"gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/locodes_updated7122021.xlsx"))
# county_district = locodes>>group_by(_.district, _.county_name)>>count(_.county_name)>>select(_.district, _.county_name)>>filter(_.county_name!='Multi-County', _.district !=53)

In [18]:
# county_info = (pd.merge(county_base, county_district, how='left', left_on= 'county_description', right_on = 'county_name'))
# county_info.drop(columns =['county_name'], axis=1, inplace=True)

In [19]:
# county_info.head()

In [20]:
# mapping1 = dict(county_info[['county_code', 'county_description']].values)
# mapping2 = dict(county_info[['county_code', 'recipient_name']].values)
# mapping3 = dict(county_info[['county_code', 'district']].values)


In [21]:
# no_locode['county_description'] = no_locode.county_code.map(mapping1)
# no_locode['recipient_name'] = no_locode.county_code.map(mapping2)
# no_locode['district'] = no_locode.county_code.map(mapping3)

In [22]:
# len(no_locode)

In [23]:
#no_locode2 = (pd.merge(no_locode, county_info, on='county_code', how='left', indicator=True))
# no_locode = no_locode.rename(columns = {"recipient_name":"implementing_agency", "county_description":"county_name"})

In [24]:
# assert lengths

In [25]:
# len(no_locode)

In [26]:
# len(locode_proj)

In [27]:
# full_df = pd.concat([locode_proj, no_locode])
    
# full_df.loc[full_df.county_name == "Statewide County", 'county_name'] = "Statewide"

In [28]:
###testing with reworked utils

## Use reworked `identify_agency` function
* now uses mapping instead of merge

In [29]:
full_df = utils.identify_agency(proj, 'summary_recipient_defined_text_field_1_value')

In [30]:
len(full_df)

1241

## Check `full_df`

In [31]:
full_df.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
907,2022-08-31,Y001,NATIONAL HIGHWAY PERF IIJA,Q101367,0417000233S,SANTA CLARA COUNTY - US 101 FROM NEAR SR 237 IN SUNNYVALE TO I-880 IN SAN JOSE CONVERT EXISTING HOV LANES TO EXPRESS LANES AND ADD A SECOND EXPRESS L,85.00,Cong Dist 17,15.00,Preliminary Engineering,"4,894,736.84",S MTC,NaN,California,4.00,Santa Clara County,NaN,NaN


In [32]:
len(full_df)

1241

In [33]:
full_df.project_number.nunique()

552

In [34]:
full_df.recipient_project_number.nunique()

549

In [35]:
full_df>>group_by(_.recipient_project_number)>>summarize(n=_.project_number.nunique())>>arrange(-_.n)

,recipient_project_number,n
79,0316000003S,2
397,0816000049S,2
0,0000001453L5,1
1,00000PE023S,1
2,0021000337S,1
...,...,...
544,1222000101L,1
545,12931559L,1
546,6500010147S,1
547,7400010045S,1


In [36]:
(full_df>>count(_.recipient_project_number)>>arrange(-_.n)).n.sum()

1240

In [37]:
(full_df>>count(_.project_number)>>arrange(-_.n)).n.sum()

1241

In [38]:
#one row does not have recipient_project_number
full_df>>filter(_.recipient_project_number.isnull())

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
1211,2022-09-23,YR10,STATE REC TRAILS ADMIN IIJA,2023000,NaN,2023 Rec Trails Project: State Parks Incurred Administrative Expenses,999.00,Cong Dist 0,44.00,Other,"241,760.00",None,NaN,California,NaN,Statewide,NaN,NaN


In [39]:
#some project title entries have differnet spellings of the same thing\
#example
full_df>>filter(_.recipient_project_number =='0316000003S')

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
666,2022-06-28,Y001,NATIONAL HIGHWAY PERF IIJA,P020195,0316000003S,"SR 20 IN COLUSA COUNTY FROM APPROX. NIAGRA RD TO WEST OF STEIDMAYER RD WIDEN BOTH SHOULDERS TO 8 FEET WITH AN RHMA STRUCTURAL SECTION, CONSTRUCT EMBA",11.00,Cong Dist 3,16.00,Right of Way,"2,000,000.00",S NON-MPO,NaN,California,3.00,Colusa County,NaN,NaN
673,2022-05-31,YS30,HIGHWAY SAFETY IMP PROG IIJA,P020197,0316000003S,"COLUSA COUNTY 1.7 MILES EAST OF COLUSA FROM NIAGARA AVENUE TO 0.3 MILE WEST OFSTEIDLMAYER ROAD RDWY EXCAVATION, COLD PLANE AC PVMT, & OVERLAY.",11.00,Cong Dist 3,17.00,Construction Engineering,"1,146,600.00",S NON-MPO,NaN,California,3.00,Colusa County,NaN,NaN
674,2022-05-31,YS30,HIGHWAY SAFETY IMP PROG IIJA,P020197,0316000003S,"COLUSA COUNTY 1.7 MILES EAST OF COLUSA FROM NIAGARA AVENUE TO 0.3 MILE WEST OFSTEIDLMAYER ROAD RDWY EXCAVATION, COLD PLANE AC PVMT, & OVERLAY.",11.00,Cong Dist 3,21.00,Safety,"5,095,070.00",S NON-MPO,NaN,California,3.00,Colusa County,NaN,NaN


* using `project_number` as unique id

#### Get one row for each project

In [40]:
# proj_unique = df.groupby(['fmis_transaction_date','program_code','project_number','recipient_project_number','project_title',
                    # 'implementing_agency', 'district', 'county_name'])['obligations_amount'].max().reset_index()

In [41]:
# merge_cols = ['fmis_transaction_date', 'program_code','project_number','recipient_project_number','project_title',
                    # 'implementing_agency', 'county_name', 'district', 'obligations_amount']

In [42]:
# proj_unique = (pd.merge(proj_unique, df, how='left', on=merge_cols))

In [43]:
# proj_unique.sample()

In [44]:
# proj_unique_cat = utils.add_description(proj_unique, 'project_title')

In [45]:
# proj_unique_cat.sample()

In [46]:
# len(proj_unique_cat>>filter(_.project_type=="Project"))

In [47]:
# proj_unique_cat_title = utils.add_new_title(proj_unique_cat, "project_method", 'project_type', 'implementing_agency')

In [48]:
# proj_unique_cat_title.sample()

## Add New Title Function

In [49]:
# #to run after first desc and title function

# def add_new_title(df, first_col_method, second_col_type, third_col_name, alt_col_name):
#     """
#     Function to add new title. 
#     Expected output example: "New Bike Lane in Eureka"
#     """
#     def return_name(df):
        
#         if (df[third_col_name] == "California") & (df[alt_col_name] == "Statewide"):
#             return (df[first_col_method] + " " + df[second_col_type] +" " + df[alt_col_name])
        
#         elif (df[third_col_name] == "California"):
#             return (df[first_col_method] + " " + df[second_col_type] + " in " + df[alt_col_name])
        
#         elif (df[third_col_name] != "California"):
#             return (df[first_col_method] + " " + df[second_col_type] + " in " + df[third_col_name])
        
#         # elif (df[third_col_name] == "Metropolitan Transportation Commission"):
#         #     return (df[first_col_method] + " " + df[second_col_type] + " in The " + df[third_col_name])

#         return df

#     df['project_name_new'] = df.apply(return_name, axis = 1)
    
#     return df

In [50]:
# proj_unique_cat_title = utils.add_new_title(proj_unique_cat_title, "project_method", "project_type", "implementing_agency", "county_name")

In [51]:
# proj_unique_cat_title.sample(3)

In [52]:
# proj_unique_cat_title = utils.update_no_matched(proj_unique_cat_title, 'project_type', 'improvement_type_description', 'implementing_agency', 'program_code_description')

In [53]:
# (proj_unique_cat_title>>filter(_.project_type=="Project")).sample(5)

In [54]:
# proj_unique_cat_title['project_name_new2'] = proj_unique_cat_title['project_name_new2'].replace('', np.NaN)

In [55]:
# proj_unique_cat_title.project_name_new2.fillna(proj_unique_cat_title.project_name_new, inplace=True)

In [56]:
# proj_unique_cat_title.drop(columns =['project_name_new'], axis=1, inplace=True)
# proj_unique_cat_title = proj_unique_cat_title.rename(columns={'project_name_new2':'project_title_new'})

## Put into full function

In [57]:
## function moved over to utils

In [58]:
utils.get_new_desc_title(full_df)

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name,project_title_new
0,2022-03-03,Y230,STBG-URBANIZED >200K IIJA,5002174,0314000222L,BELL AVE. FROM BOLLENBACHER AVE. TO ASTORIA ST ROAD REHABILITATION (TC),67.00,Cong Dist 6,4.00,4R - No Added Capacity,"2,841,397.00",L5002SACOG,"5,002.00",Sacramento,3.00,Sacramento County,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,Road Rehabiliation in Sacramento
1,2022-03-03,Y230,STBG-URBANIZED >200K IIJA,5002174,0314000222L,BELL AVE. FROM BOLLENBACHER AVE. TO ASTORIA ST ROAD REHABILITATION (TC),67.00,Cong Dist 6,17.00,Construction Engineering,"255,927.00",L5002SACOG,"5,002.00",Sacramento,3.00,Sacramento County,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,Road Rehabiliation in Sacramento
2,2022-08-26,Y400,CONGESTION MITIGATION IIJA,5002178,0315000001L,"SOUTH SACRAMENTO PARKWAY TRAIL- WEST. IN THE CITY OF SACRAMENTO, ADJACENT TO NORTHBOUND INTERSTATE 5, SOUTH OF POCKET ROAD/MEADOWVIEW ROAD AND FREEPO",67.00,Cong Dist 6,17.00,Construction Engineering,"50,000.00",L5002SACOG,"5,002.00",Sacramento,3.00,Sacramento County,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,Facilities for Pedestrians and Bicycles in Sacramento
3,2022-08-26,Y400,CONGESTION MITIGATION IIJA,5002178,0315000001L,"SOUTH SACRAMENTO PARKWAY TRAIL- WEST. IN THE CITY OF SACRAMENTO, ADJACENT TO NORTHBOUND INTERSTATE 5, SOUTH OF POCKET ROAD/MEADOWVIEW ROAD AND FREEPO",67.00,Cong Dist 6,28.00,Facilities for Pedestrians and Bicycles,"508,614.00",L5002SACOG,"5,002.00",Sacramento,3.00,Sacramento County,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,Facilities for Pedestrians and Bicycles in Sacramento
4,2022-07-15,Y400,CONGESTION MITIGATION IIJA,5002189,0316000158L,ALONG UNUSED RAIL CORRIDOR NEAR FREEPORT BLVD FROM SUTTERVILLE ROAD TO SOUTH OF MEADOWVIEW/POCKET ROAD (4.5 MILES) RAILS TO TRAILS PROJECT ; NEW CLAS,67.00,Cong Dist 6,28.00,Facilities for Pedestrians and Bicycles,"4,359,000.00",L5002SACOG,"5,002.00",Sacramento,3.00,Sacramento County,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,Facilities for Pedestrians and Bicycles in Sacramento
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,2022-09-09,YS30,HIGHWAY SAFETY IMP PROG IIJA,5801081,0416000125S,ALAMEDA COUNTY NEAR LIVERMORE FROM GRANT LINE ROAD UNDERCROSSING TO 0.1 MILE EAST OF NORTH FLYNN ROAD INSTALL LIGHTING SYSTEM,1.00,Cong Dist 15,21.00,Safety,"4,165,800.00",S MTC,NaN,California,4.00,Alameda County,NaN,NaN,Install Lighting in Alameda County
1247,2022-09-09,YS30,HIGHWAY SAFETY IMP PROG IIJA,5801081,0416000125S,ALAMEDA COUNTY NEAR LIVERMORE FROM GRANT LINE ROAD UNDERCROSSING TO 0.1 MILE EAST OF NORTH FLYNN ROAD INSTALL LIGHTING SYSTEM,1.00,Cong Dist 15,22.00,Rail/Hwy Crossing,"18,900.00",S MTC,NaN,California,4.00,Alameda County,NaN,NaN,Install Lighting in Alameda County
1248,2022-09-09,YS30,HIGHWAY SAFETY IMP PROG IIJA,5801081,0416000125S,ALAMEDA COUNTY NEAR LIVERMORE FROM GRANT LINE ROAD UNDERCROSSING TO 0.1 MILE EAST OF NORTH FLYNN ROAD INSTALL LIGHTING SYSTEM,1.00,Cong Dist 15,42.00,Training,"3,000.00",S MTC,NaN,California,4.00,Alameda County,NaN,NaN,Install Lighting in Alameda County
1249,2022-09-09,YS32,SEC 164 PENALTIES HSIP IIJA,5801081,0416000125S,ALAMEDA COUNTY NEAR LIVERMORE FROM GRANT LINE ROAD UNDERCROSSING TO 0.1 MILE EAST OF NORTH FLYNN ROAD INSTALL LIGHTING SYSTEM,1.00,Cong Dist 15,17.00,Construction Engineering,"288,000.00",S MTC,NaN,California,4.00,Alameda County,NaN,NaN,Install Lighting in Alameda County


In [59]:
full_df>>filter(_.project_title_new.str.startswith('Project'))>>select(_.project_title, _.project_title_new)

,project_title,project_title_new
556,SOLANO COUNTY DEVELOP COUNTYWIDE CONNECTED MOBILITY IMPLEMENTATION PLAN TO ADDRESS HOW SOLANO COUNTY ADDRESSES THE RECOMMENDATIONS OF BLUE RIBBON T,Project Planning in Solano Transportation Authority
1151,IN SHASTA COUNTY ABOUT 5 MILES NORTH OF SHASTA LAKE CITY FROM 1.0 MILE NORTH OF FAWNDALE OVERCROSSING TO 0.2 MILE NORTH OF TURNTABLE BAY ROAD OVERCRO,Project Planning in California


In [60]:
full_df.tail()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name,project_title_new
1246,2022-09-09,YS30,HIGHWAY SAFETY IMP PROG IIJA,5801081,0416000125S,ALAMEDA COUNTY NEAR LIVERMORE FROM GRANT LINE ROAD UNDERCROSSING TO 0.1 MILE EAST OF NORTH FLYNN ROAD INSTALL LIGHTING SYSTEM,1.00,Cong Dist 15,21.00,Safety,"4,165,800.00",S MTC,NaN,California,4.00,Alameda County,NaN,NaN,Install Lighting in Alameda County
1247,2022-09-09,YS30,HIGHWAY SAFETY IMP PROG IIJA,5801081,0416000125S,ALAMEDA COUNTY NEAR LIVERMORE FROM GRANT LINE ROAD UNDERCROSSING TO 0.1 MILE EAST OF NORTH FLYNN ROAD INSTALL LIGHTING SYSTEM,1.00,Cong Dist 15,22.00,Rail/Hwy Crossing,"18,900.00",S MTC,NaN,California,4.00,Alameda County,NaN,NaN,Install Lighting in Alameda County
1248,2022-09-09,YS30,HIGHWAY SAFETY IMP PROG IIJA,5801081,0416000125S,ALAMEDA COUNTY NEAR LIVERMORE FROM GRANT LINE ROAD UNDERCROSSING TO 0.1 MILE EAST OF NORTH FLYNN ROAD INSTALL LIGHTING SYSTEM,1.00,Cong Dist 15,42.00,Training,"3,000.00",S MTC,NaN,California,4.00,Alameda County,NaN,NaN,Install Lighting in Alameda County
1249,2022-09-09,YS32,SEC 164 PENALTIES HSIP IIJA,5801081,0416000125S,ALAMEDA COUNTY NEAR LIVERMORE FROM GRANT LINE ROAD UNDERCROSSING TO 0.1 MILE EAST OF NORTH FLYNN ROAD INSTALL LIGHTING SYSTEM,1.00,Cong Dist 15,17.00,Construction Engineering,"288,000.00",S MTC,NaN,California,4.00,Alameda County,NaN,NaN,Install Lighting in Alameda County
1211,2022-09-23,YR10,STATE REC TRAILS ADMIN IIJA,2023000,NaN,2023 Rec Trails Project: State Parks Incurred Administrative Expenses,999.00,Cong Dist 0,44.00,Other,"241,760.00",None,NaN,California,NaN,Statewide,NaN,NaN,Recreational Trails Statewide


### Reworking of title function application

In [61]:
#redo

In [62]:
#proj_one_row = full_df.groupby(['project_number', 'project_title'])['obligations_amount'].max().reset_index()

In [63]:
#len(proj_one_row)

In [64]:
#merge_cols = ['project_number','project_title','obligations_amount']


In [65]:
#proj_unique = (pd.merge(proj_one_row, full_df, how='left', on=merge_cols))

In [66]:
# proj_unique>>group_by(_.project_number, _.project_title)>>count(_.obligations_amount)>>arrange(_.n)

* duplicates here have different congressional districts- so we have more entries

In [67]:
# proj_unique_cat = utils.add_description(proj_unique, 'project_title')

In [68]:
# proj_unique_cat_title = utils.add_new_title(proj_unique_cat, "project_method", "project_type", "implementing_agency", "county_name")


In [69]:
# proj_unique_cat_title = utils.update_no_matched(proj_unique_cat_title, 'project_type', 'improvement_type_description', 'implementing_agency', 'program_code_description')


In [70]:
# proj_unique_cat_title['project_name_new2'] = proj_unique_cat_title['project_name_new2'].replace('', np.NaN)


In [71]:
# proj_unique_cat_title.project_name_new2.fillna(proj_unique_cat_title.project_name_new, inplace=True)


In [72]:
# proj_unique_cat_title.drop(columns =['project_name_new'], axis=1, inplace=True)
# proj_unique_cat_title = proj_unique_cat_title.rename(columns={'project_name_new2':'project_title_new'})

In [73]:
# proj_unique_cat_title.sample()

In [74]:
#proj_unique_cat_title = get_new_desc_title(full_df)

In [75]:
#proj_unique_cat_title.sample()

In [76]:
# proj_title_mapping = dict(proj_unique_cat_title[['project_number', 'project_title_new']].values)


In [77]:
# full_df['project_title_new'] = full_df.project_number.map(proj_title_mapping)


## Check new Title

In [78]:
full_df.sample(5)

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name,project_title_new
439,2022-08-10,Y233,STBG IIJA OFF-SYSTEM BRIDGE,5939083,1012000171L,LOS CERRITOS ROAD OVER DRY CREEK (BRIDGE 39C0225) BRIDGE REPLACEMENT (TC),47.00,Cong Dist 16,11.00,Bridge Replacement - No Added Capacity,"56,163.53",L5939MCAG,"5,939.00",Merced County,10.00,Merced County,Merced County Association of Governments,Merced County Association Of Goverments,Bridge Replacement in Merced County
422,2022-04-04,Y233,STBG IIJA OFF-SYSTEM BRIDGE,5937058,04923198L,ALAMITOS RD. BRIDGE @ ALAMITOS CK (37C0159) BRIDGE REPLACEMENT/SEISMIC RET (TC),85.00,Cong Dist 19,11.00,Bridge Replacement - No Added Capacity,"7,909,832.00",L5937MTC,"5,937.00",Santa Clara County,4.00,Santa Clara County,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Bridge Replacement in Santa Clara County
137,2022-06-06,Y001,NATIONAL HIGHWAY PERF IIJA,5068037,0200020171L,CANYON RD @ ACID CANAL. BR. # 06C0307 BRIDGE REPLACEMENT,89.00,Cong Dist 1,16.00,Right of Way,"69,053.00",L5068SHASTA,"5,068.00",Redding,2.00,Shasta County,Shasta County Regional Transportation Planning Agency,Shasta County Regional Transportation Planning Age,Replace Bridge in Redding
1035,2022-06-23,Y001,NATIONAL HIGHWAY PERF IIJA,X077028,1016000026S,SAN JOAQUIN COUNTY IN STOCKTON ON ROUTE 4 FROM 0.2 MILE EAST OF FRESNO AVENUE TO EL DORADO STREET AND ON ROUTE 5 FROM TAYLOR STREET OVERHEAD TO STOC,77.00,Cong Dist 9,17.00,Construction Engineering,"4,499,000.00",S SJCOG,NaN,California,10.00,San Joaquin County,NaN,NaN,Overhead in San Joaquin County
402,2022-07-14,Y233,STBG IIJA OFF-SYSTEM BRIDGE,5930067,1000020540L,SCHADD ROAD OVER FOREST CREEK (BRIDGE 30C0084) BRIDGE REPLACEMENT (TC),9.00,Cong Dist 4,11.00,Bridge Replacement - No Added Capacity,"592,838.00",L5930NON-MPO,"5,930.00",Calaveras County,10.00,Calaveras County,Calaveras Council of Governments,NON-MPO,Bridge Replacement in Calaveras County


In [79]:
full_df>>filter(_.project_title_new.isna())

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name,project_title_new


In [80]:
#checking the length of entries here to make sure
len(full_df)

1241

In [88]:
full_df>>count(_.project_title_new)>>arrange(-_.n)

,project_title_new,n
354,Road Restoration & Rehabilitation in California,112
361,Safety Improvements in California,91
27,Planning Statewide,43
79,Bridge Replacement in California,27
64,Added Roadway Capacity in California,22
...,...,...
383,Upgrade Bike Lanes in Galt,1
384,Upgrade Erosion Countermeasures in Ventura County,1
388,Upgrade Guardrails in Jurupa Valley,1
400,Upgrade Traffic Signals in San Bernardino,1


In [92]:
len(full_df>>filter(_.project_title_new.str.contains("in California")))

335

In [91]:
full_df>>filter(_.project_title_new.str.contains("in California"))>>count(_.project_title_new)

,project_title_new,n
0,Added Roadway Capacity in California,22
1,Bridge Rehabilitation in California,9
2,Bridge Replacement in California,27
3,Construction Engineering Projects in California,13
4,Landscaping and Other Scenic Beautification in California,1
5,Maintenance Resurfacing in California,13
6,Mitigation of Water Pollution due to Highway Runoff in California,12
7,National Highway System Support in California,12
8,Preliminary Engineering Projects in California,11
9,Project Planning in California,1


In [89]:
full_df>>filter(_.project_title_new=="Road Restoration & Rehabilitation in California")

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name,project_title_new
584,2022-05-05,Y001,NATIONAL HIGHWAY PERF IIJA,6801946,0414000019S,ALAMEDA COUNTY AT SUNOL AND IN PLEASANTON AND DUBLIN PRECAST CONCRETE PAVEMENT AND HMA,1.00,Cong Dist 15,6.00,4R - Restoration & Rehabilitation,"66,377,100.00",S MTC,NaN,California,4.00,Alameda County,NaN,NaN,Road Restoration & Rehabilitation in California
585,2022-05-05,Y001,NATIONAL HIGHWAY PERF IIJA,6801946,0414000019S,ALAMEDA COUNTY AT SUNOL AND IN PLEASANTON AND DUBLIN PRECAST CONCRETE PAVEMENT AND HMA,1.00,Cong Dist 15,17.00,Construction Engineering,"8,607,500.00",S MTC,NaN,California,4.00,Alameda County,NaN,NaN,Road Restoration & Rehabilitation in California
586,2022-05-05,Y001,NATIONAL HIGHWAY PERF IIJA,6801946,0414000019S,ALAMEDA COUNTY AT SUNOL AND IN PLEASANTON AND DUBLIN PRECAST CONCRETE PAVEMENT AND HMA,1.00,Cong Dist 15,42.00,Training,"7,600.00",S MTC,NaN,California,4.00,Alameda County,NaN,NaN,Road Restoration & Rehabilitation in California
587,2022-05-10,Y001,NATIONAL HIGHWAY PERF IIJA,8051641,1117000037S,"IN SAN DIEGO COUNTY IN CHULA VISTA, FROM 0.3 MILE SOUTH OF MAIN STREET UNDERCROSSING TO PALOMAR STREET OVERCROSSING CONVERT POTABLE IRRIGATION SYSTEM",73.00,Cong Dist 51,6.00,4R - Restoration & Rehabilitation,"279,235.77",S SANDAG,NaN,California,11.00,San Diego County,NaN,NaN,Road Restoration & Rehabilitation in California
588,2022-05-10,Y001,NATIONAL HIGHWAY PERF IIJA,8051641,1117000037S,"IN SAN DIEGO COUNTY IN CHULA VISTA, FROM 0.3 MILE SOUTH OF MAIN STREET UNDERCROSSING TO PALOMAR STREET OVERCROSSING CONVERT POTABLE IRRIGATION SYSTEM",73.00,Cong Dist 51,17.00,Construction Engineering,"140,606.12",S SANDAG,NaN,California,11.00,San Diego County,NaN,NaN,Road Restoration & Rehabilitation in California
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,2022-06-21,Y001,NATIONAL HIGHWAY PERF IIJA,2801153,0416000024S,"SANTA CLARA COUNTY IN AND NEAR SAN JOSE FROM S280-N101/101 SEPARATION TO ROUTE 17/280 SEPERATION MINOR CONCRETE, MGS RAILING, HMA AND HMA DIKE.",85.00,Cong Dist 18,17.00,Construction Engineering,"202,953.35",S MTC,NaN,California,4.00,Santa Clara County,NaN,NaN,Road Restoration & Rehabilitation in California
1231,2022-06-21,Y001,NATIONAL HIGHWAY PERF IIJA,2801153,0416000024S,"SANTA CLARA COUNTY IN AND NEAR SAN JOSE FROM S280-N101/101 SEPARATION TO ROUTE 17/280 SEPERATION MINOR CONCRETE, MGS RAILING, HMA AND HMA DIKE.",85.00,Cong Dist 18,42.00,Training,563.55,S MTC,NaN,California,4.00,Santa Clara County,NaN,NaN,Road Restoration & Rehabilitation in California
1232,2022-06-21,Y001,NATIONAL HIGHWAY PERF IIJA,2801153,0416000024S,"SANTA CLARA COUNTY IN AND NEAR SAN JOSE FROM S280-N101/101 SEPARATION TO ROUTE 17/280 SEPERATION MINOR CONCRETE, MGS RAILING, HMA AND HMA DIKE.",85.00,Cong Dist 19,6.00,4R - Restoration & Rehabilitation,"2,074,422.77",S MTC,NaN,California,4.00,Santa Clara County,NaN,NaN,Road Restoration & Rehabilitation in California
1233,2022-06-21,Y001,NATIONAL HIGHWAY PERF IIJA,2801153,0416000024S,"SANTA CLARA COUNTY IN AND NEAR SAN JOSE FROM S280-N101/101 SEPARATION TO ROUTE 17/280 SEPERATION MINOR CONCRETE, MGS RAILING, HMA AND HMA DIKE.",85.00,Cong Dist 19,17.00,Construction Engineering,"337,246.65",S MTC,NaN,California,4.00,Santa Clara County,NaN,NaN,Road Restoration & Rehabilitation in California


## Export data frame with new titles

In [81]:
#GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [82]:
#test_df = utils.title_column_names(full_df)

In [83]:
#test_df.to_csv(f"{GCS_FILE_PATH}/FMIS_projects_all.csv")

## adding locations

In [85]:
full_df.sample(4)

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name,project_title_new
565,2022-05-12,Y001,NATIONAL HIGHWAY PERF IIJA,6507003,0817000123L,"MOUNT VERNON AVENUE OVERHEAD AT THE BURLINGTON NORTHERN SANTA FE RAILROAD YARD FROM KING STREET TO 5TH STREET, BR. NO. 54C-0066 BRIDGE REPLACEMENT",71.00,Cong Dist 31,43.00,Utilities,"22,132.00",L6507SCAG,"6,507.00",San Bernardino County Transportation Authority,8.00,San Bernardino County,NON-RTPA,Southern California Association Of Governments,Replace Bridge in San Bernardino County Transportation Authority
219,2022-08-11,YS30,HIGHWAY SAFETY IMP PROG IIJA,5205027,0818800003L,BASELINE ROAD - WILLOW AVENUE TO MERIDIAN AVENUE CONSTRUCT A 12' WIDE RAISED CENTER MEDIAN AND ADA RAMP.,71.00,Cong Dist 31,21.00,Safety,"1,070,266.00",L5205SCAG,"5,205.00",Rialto,8.00,San Bernardino County,San Bernardino Associated Governments,Southern California Association Of Governments,Construct Raised Median in Rialto
284,2022-09-12,Y306,TRANS ALTERN 50K-200K POP IIJA,5382038,1120000031L,THE TOWNSITE DRIVE COMPLETE STREETS PROJECT IS LOCATED IN THE CITY OF VISTA ALONG THE SEGMENT OF NORTH DRIVE FROM JUST NORTHWEST OF THE INTERSECTION,73.00,Cong Dist 49,28.00,Facilities for Pedestrians and Bicycles,"3,232,230.00",L5382SANDAG,"5,382.00",Vista,11.00,San Diego County,San Diego Association of Governments,San Diego Association Of Governments,Complete Streets in Vista
781,2022-04-29,Y001,NATIONAL HIGHWAY PERF IIJA,P070143,0321000091S,IN YUBA AND BUTTE COUNTIES ABOUT 11 MILES NORTH OF MARYSVILLE FROM 0.8 MILE NORTH OF CHANDLER ROAD TO 0.2 MILE SOUTH OF EAST GRIDLEY ROAD WIDEN ROADW,115.00,Cong Dist 3,42.00,Training,200.42,S BCAG,NaN,California,3.00,Yuba County,NaN,NaN,Widen Road in Butte County
